In [1]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from keras.preprocessing.sequence import pad_sequences

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# 데이터 불러오기

- 주가변동률에 따라 긍부정도를 labeling한 GS건설, KB금융, 아모레퍼시픽, 삼성전자, 이마트 데이터 셋을 이용하여 훈련

- 순서
    1. 데이터 불러오기
    2. 데이터 분리
    3. Title 의 토큰화 및 불용어 제거
    4. Label 의 원핫인코딩
    5. 길이가 다른 title의 정형화
    6. LSTM을 통한 딥러닝
    7. 정확도 확인

## 데이터셋 구성
- Train Dataset : kospis 2000개 합친 데이터의 80%
- Test Dataset : kospis 2000개 합친 데이터의 20%

In [66]:
# 각 종목별 2000개 데이터 추출하여 train dataframe 만들기

kospis = ['GS건설','KB금융','아모레퍼시픽','삼성전자','이마트'] 
train_df = pd.DataFrame(columns = ['Date', 'Category', 'Title', 'Label'])

for kospi in kospis:
    temp_df = pd.read_csv('../../Data/PriceChange/{}_roc_traindata.csv'.format(kospi))
    temp_df = shuffle(temp_df)
    train_df = train_df.append(temp_df[:2000])

In [109]:
X = train_df.Title
y = train_df.Label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2, stratify=y)

In [110]:
# 불용어 제거 및 단어 분리
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', \
              '하다', '까지','…','?','.',',','!','"',"'",'로','고','(',')','[',']','ㄹ까']

import konlpy
from konlpy.tag import Okt

okt = Okt()
X_train_token = []
for sentence in X_train:
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train_token.append(temp_X)

X_test_token = []
for sentence in X_test:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test_token.append(temp_X)

In [ ]:
X

### 단어길이 추출

In [111]:
from keras.preprocessing.text import Tokenizer
max_words = 15000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train_token) 
X_train_token = tokenizer.texts_to_sequences(X_train_token) 
X_test_token = tokenizer.texts_to_sequences(X_test_token)

In [112]:
li_train = []
for i in range(len(X_train_token)):
   li_train.append(max(X_train_token[i]))


li_test = []
for i in range(len(X_test_token)):
   li_test.append(max(X_test_token[i]))

print(max(li_train), max(li_test))

11508 11502


In [80]:
li_train = []
# li_train.append(1)
# li_train.append(2)
for i in range(len(X_train_token)):
   li_train.append(len(X_train_token[i]))


li_test = []
# li_train.append(1)
# li_train.append(2)
for i in range(len(X_test_token)):
   li_test.append(len(X_test_token[i]))
print(max(li_train), max(li_test))

27 25


### One-Hot encoding

In [113]:
y_train_encode = []
y_test_encode = []
for i in range(len(y_train)):
    if y_train.iloc[i] == 1:
        y_train_encode.append([0, 1]) 
    else:
        y_train_encode.append([1, 0]) 
        
for i in range(len(y_test)):
    if y_test.iloc[i] == 1:
        y_test_encode.append([0, 1]) 
    else:
        y_test_encode.append([1, 0]) 
        
y_train_encode=np.array(y_train_encode)
y_test_encode=np.array(y_test_encode)

### max_len 20 to 100으로 변경 ; 단어 길이 고려

In [86]:
max_len = 30 # pad_sequesces이용, 전체 데이터의 길이를 20로 맞춘다
X_train = pad_sequences(X_train_token, maxlen=max_len)
X_test = pad_sequences(X_test_token, maxlen=max_len)

### 모델 저장1
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


MODEL_DIR = './ModelCheckpoint/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
mdpath = './ModelCheckpoint/{epoch:02d}-{val_loss:.4f}.hdf5'
mc = ModelCheckpoint(filepath=mdpath,monitor='val_acc',mode='max',verbose=1,save_best_only=True)
es = EarlyStopping(monitor='val_loss',patience=10,mode='min',verbose=1)

model = Sequential()
model.add(Embedding(max_words, 64))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train_cata, epochs=10, batch_size=10, callbacks=[es,mc], validation_split=0.1)

### 모델 저장2
from keras.models import load_model
##### 모델 컴퓨터에 저장
model.save('모델이름')
##### 모델 불러오기
del model # 테스트를 위해 메모리 내의 모델을 삭제
model = load_model('모델명')

### LSTM 모델

### earlystopping과 modelcheckpoint 추가
### embedding 64로 변경

In [115]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=10)
mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)

model = Sequential()
model.add(Embedding(max_words, 64))
model.add(LSTM(128,activation='tanh'))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train_encode, epochs=20, batch_size=10, callbacks=[es,mc], validation_split=0.1)

print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test_encode)[1]*100))

Epoch 1/20


UnimplementedError:  Cast string to float is not supported
	 [[node sequential_21/Cast (defined at <ipython-input-115-6550f11d33eb>:10) ]] [Op:__inference_train_function_96753]

Function call stack:
train_function


In [107]:
print(len(X_train), len(y_train_encode))

8000 8000


In [114]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

model6 = Sequential()
model6.add(Embedding(max_words, 100))
model6.add(Dropout(0.5))
model6.add(Conv1D(64, 5, padding='valid', activation='relu', strides=1))
model6.add(MaxPooling1D(pool_size=4))
model6.add(LSTM(55))
model6.add(Dense(2, activation='relu'))

model6.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model6.fit(X_train, y_train_encode, epochs=20, batch_size=100, callbacks=[es,mc], validation_data=(X_test, y_test_encode))

print("\n 테스트 정확도 : {:.2f}%".format(model6.evaluate(X_test,y_test_encode)[1]*100))

Epoch 1/20


ValueError: in user code:

    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
 

model.save('M1_61.21%')

## KFOLD 검증

In [78]:
#seed 값 설정
seed = 1
np.random.seed(seed)
#tf.set_random_seed(seed)

In [79]:
X=X_train
Y=y_train_cata

In [80]:
#10개의 파일로 쪼갬
n_fold = 15

In [81]:
skf =KFold(n_splits=n_fold, shuffle=True,random_state=seed)

In [82]:
#빈 accuracy 배열
accuracy =[]

In [84]:
#모델의 설정, 컴파일, 실행
for train, test in skf.split(X,Y):
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
    es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=4)
    mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)

    model = Sequential()
    model.add(Embedding(max_words, 64))
    model.add(LSTM(128,activation='tanh'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X[train],Y[train], epochs=10, batch_size=10, callbacks=[es,mc], validation_split=0.1)
    k_accuracy = '%.4f'%(model.evaluate(X[test],Y[test])[1])
    accuracy.append(k_accuracy)

Epoch 1/10
103/103 [==============================] - 5s 50ms/step - loss: 1.0034 - accuracy: 0.5253 - val_loss: 0.9877 - val_accuracy: 0.5439
Epoch 2/10
103/103 [==============================] - 5s 45ms/step - loss: 0.9308 - accuracy: 0.5828 - val_loss: 0.7731 - val_accuracy: 0.7105
Epoch 3/10
103/103 [==============================] - 5s 45ms/step - loss: 0.8479 - accuracy: 0.6072 - val_loss: 0.7933 - val_accuracy: 0.7281
Epoch 4/10
103/103 [==============================] - 5s 45ms/step - loss: 0.8140 - accuracy: 0.6170 - val_loss: 0.7442 - val_accuracy: 0.7193
Epoch 5/10
103/103 [==============================] - 5s 45ms/step - loss: 0.7925 - accuracy: 0.6140 - val_loss: 0.7682 - val_accuracy: 0.6930
Epoch 6/10
103/103 [==============================] - 5s 45ms/step - loss: 0.7822 - accuracy: 0.6248 - val_loss: 0.7500 - val_accuracy: 0.6930
Epoch 7/10
103/103 [==============================] - 5s 45ms/step - loss: 0.7663 - accuracy: 0.6384 - val_loss: 0.8206 - val_accuracy: 0.6667

Epoch 4/10
103/103 [==============================] - 6s 56ms/step - loss: 0.8074 - accuracy: 0.6160 - val_loss: 0.7603 - val_accuracy: 0.7105
Epoch 5/10
103/103 [==============================] - 6s 55ms/step - loss: 0.7865 - accuracy: 0.6257 - val_loss: 0.7794 - val_accuracy: 0.6930
Epoch 6/10
103/103 [==============================] - 6s 55ms/step - loss: 0.7737 - accuracy: 0.6404 - val_loss: 0.8266 - val_accuracy: 0.7018
Epoch 7/10
103/103 [==============================] - 6s 54ms/step - loss: 0.7617 - accuracy: 0.6433 - val_loss: 0.8009 - val_accuracy: 0.7018
Epoch 8/10
103/103 [==============================] - 6s 54ms/step - loss: 0.7571 - accuracy: 0.6452 - val_loss: 0.8409 - val_accuracy: 0.6579
Epoch 00008: early stopping
3/3 [==============================] - 0s 12ms/step - loss: 0.9072 - accuracy: 0.5610
Epoch 1/10
103/103 [==============================] - 5s 52ms/step - loss: 1.0028 - accuracy: 0.5292 - val_loss: 0.9903 - val_accuracy: 0.5614
Epoch 2/10
103/103 [========

Epoch 8/10
103/103 [==============================] - 7s 67ms/step - loss: 0.7404 - accuracy: 0.6394 - val_loss: 0.8264 - val_accuracy: 0.6667
Epoch 9/10
103/103 [==============================] - 7s 67ms/step - loss: 0.7324 - accuracy: 0.6686 - val_loss: 0.8056 - val_accuracy: 0.6842
Epoch 00009: early stopping
3/3 [==============================] - 0s 13ms/step - loss: 0.9027 - accuracy: 0.5488
Epoch 1/10
103/103 [==============================] - 6s 61ms/step - loss: 1.0142 - accuracy: 0.5341 - val_loss: 0.9939 - val_accuracy: 0.5304
Epoch 2/10
103/103 [==============================] - 6s 57ms/step - loss: 0.9069 - accuracy: 0.5556 - val_loss: 0.8102 - val_accuracy: 0.7304
Epoch 3/10
103/103 [==============================] - 6s 57ms/step - loss: 0.8412 - accuracy: 0.5955 - val_loss: 0.7479 - val_accuracy: 0.6609
Epoch 4/10
103/103 [==============================] - 6s 57ms/step - loss: 0.8065 - accuracy: 0.6248 - val_loss: 0.7515 - val_accuracy: 0.6870
Epoch 5/10
103/103 [========

Epoch 5/10
103/103 [==============================] - 6s 59ms/step - loss: 0.7827 - accuracy: 0.6365 - val_loss: 0.7314 - val_accuracy: 0.7217
Epoch 6/10
103/103 [==============================] - 6s 59ms/step - loss: 0.7905 - accuracy: 0.6394 - val_loss: 0.8992 - val_accuracy: 0.5478
Epoch 7/10
103/103 [==============================] - 6s 59ms/step - loss: 0.7934 - accuracy: 0.6218 - val_loss: 0.7652 - val_accuracy: 0.6696
Epoch 8/10
103/103 [==============================] - 6s 60ms/step - loss: 0.7446 - accuracy: 0.6559 - val_loss: 0.7676 - val_accuracy: 0.7043
Epoch 9/10
103/103 [==============================] - 6s 60ms/step - loss: 0.7324 - accuracy: 0.6618 - val_loss: 0.8271 - val_accuracy: 0.6609
Epoch 00009: early stopping
3/3 [==============================] - 0s 13ms/step - loss: 0.9885 - accuracy: 0.5926
Epoch 1/10
103/103 [==============================] - 6s 61ms/step - loss: 1.0028 - accuracy: 0.5263 - val_loss: 0.9828 - val_accuracy: 0.5391
Epoch 2/10
103/103 [========

Epoch 8/10
103/103 [==============================] - 6s 63ms/step - loss: 0.7409 - accuracy: 0.6481 - val_loss: 0.8573 - val_accuracy: 0.6435
Epoch 9/10
103/103 [==============================] - 6s 62ms/step - loss: 0.7314 - accuracy: 0.6452 - val_loss: 0.8596 - val_accuracy: 0.6435
Epoch 00009: early stopping
3/3 [==============================] - 0s 14ms/step - loss: 0.8672 - accuracy: 0.6420
Epoch 1/10
103/103 [==============================] - 6s 63ms/step - loss: 1.0031 - accuracy: 0.5302 - val_loss: 0.9898 - val_accuracy: 0.5826
Epoch 2/10
103/103 [==============================] - 6s 61ms/step - loss: 0.9033 - accuracy: 0.5702 - val_loss: 0.7544 - val_accuracy: 0.7217
Epoch 3/10
103/103 [==============================] - 6s 60ms/step - loss: 0.8485 - accuracy: 0.5877 - val_loss: 0.7119 - val_accuracy: 0.7043
Epoch 4/10
103/103 [==============================] - 6s 60ms/step - loss: 0.8148 - accuracy: 0.6062 - val_loss: 0.7884 - val_accuracy: 0.6696
Epoch 5/10
103/103 [========

In [85]:
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test_cata)[1]*100))

9/9 [==============================] - 0s 19ms/step - loss: 0.9478 - accuracy: 0.5836

 테스트 정확도 : 58.36%


## Learning Rate 조정

In [41]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD, RMSprop, Adagrad, Adam

In [88]:
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=10)
mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)
opt ='adam'

model = Sequential()
model.add(Embedding(max_words, 64))
model.add(LSTM(128,activation='tanh'))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train_encode, epochs=20, batch_size=10, callbacks=[es,mc], validation_split=0.1)
    
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test_encode)[1]*100))

Epoch 1/20
720/720 [==============================] - 13s 18ms/step - loss: 0.6926 - accuracy: 0.5069 - val_loss: 0.6929 - val_accuracy: 0.5113
Epoch 2/20
720/720 [==============================] - 13s 18ms/step - loss: 0.6025 - accuracy: 0.6853 - val_loss: 0.7464 - val_accuracy: 0.5200
Epoch 3/20
720/720 [==============================] - 13s 18ms/step - loss: 0.3625 - accuracy: 0.8413 - val_loss: 0.9289 - val_accuracy: 0.5263
Epoch 4/20
720/720 [==============================] - 13s 19ms/step - loss: 0.1781 - accuracy: 0.9317 - val_loss: 1.3318 - val_accuracy: 0.5200
Epoch 5/20
720/720 [==============================] - 13s 18ms/step - loss: 0.1022 - accuracy: 0.9629 - val_loss: 1.4832 - val_accuracy: 0.5387
Epoch 6/20
720/720 [==============================] - 13s 18ms/step - loss: 0.0691 - accuracy: 0.9726 - val_loss: 1.9646 - val_accuracy: 0.5200
Epoch 7/20
720/720 [==============================] - 13s 18ms/step - loss: 0.0501 - accuracy: 0.9799 - val_loss: 2.4285 - val_accuracy:

In [95]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.layers import Dense, Dropout, Activation

model2=models.Sequential()

model2.add(layers.Dense(100, activation='relu'))
model2.add(layers.Dense(80, activation='relu'))
model2.add(layers.Dense(60, activation='relu'))
model2.add(layers.Dense(40, activation='relu'))
model2.add(layers.Dense(20, activation='relu'))

model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))


model2.compile(optimizer=optimizers.RMSprop(lr=0.0001)
             ,loss=losses.binary_crossentropy
             ,metrics=['accuracy'])

history = model2.fit(X_train, y_train_encode, epochs=300, batch_size=10, callbacks=[es,mc], validation_split=0.1)
print("\n 테스트 정확도 : {:.2f}%".format(model2.evaluate(X_test,y_test_encode)[1]*100))
# hist = model.fit(train_x,train_y,epochs=300,batch_size=500, validation_data=(test_x, test_y), verbose = 2)

Epoch 1/300


ValueError: in user code:

    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
 

In [92]:
model2.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 100)               3100      
_________________________________________________________________
dense_22 (Dense)             (None, 80)                8080      
_________________________________________________________________
dense_23 (Dense)             (None, 60)                4860      
_________________________________________________________________
dense_24 (Dense)             (None, 40)                2440      
_________________________________________________________________
dense_25 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_26 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_27 (Dense)             (None, 1)               

In [48]:
from matplotlib import pyplot

In [98]:
#optimizer
# adam = Adam()
# sgd=SGD(lr=0.01,momentum=0.9,decay=0.01)
# ad = Adagrad()
# rmsprop = RMSprop()
#fit a model and plot learning curve
def fit_model(X_train,y_train_cata,X_test,y_test_cata,optimizer):
    es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=10)
    mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)
    #define model
    model = Sequential()
    model.add(Embedding(max_words,64))
    model.add(LSTM(128,activation='tanh'))
    model.add(Dense(50, input_dim=2,activation='relu'))
    model.add(Dense(3, activation='softmax'))
    #compile model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    #fit model
    history = model.fit(X_train, y_train_cata, epochs=20, batch_size=10, callbacks=[es,mc],validation_split=0.1,verbose=0.1)
    #print 테스트 정확도
    print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test_cata)[1]*100))
    #plot learning curves
    pyplot.plot(history.history['accuracy'],label='train')
    pyplot.plot(history.history['val_accuracy'],label='test')
    pyplot.title('opt='+optimizer,pad=80)

In [102]:
# adam = Adam()
fit_model(X_train,y_train_encode,X_test,y_test_encode,'adam')

Epoch 1/20


ValueError: in user code:

    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
 

## 베이지안 최적화

In [5]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11689 sha256=984db3c61c973454b5b37d87832caaa5959ad9dc65c1f371823f920efec4bd7b
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\fd\9b\71\f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [6]:
from bayes_opt import BayesianOptimization

In [7]:
#BaysianOptimization 객체생성
BO_lgb = BayesianOptimization(성능 평가를 수행할 함수, 테스트 수행을 위한 하이퍼 파라미터 Dic,random_state=0)

SyntaxError: invalid syntax (<ipython-input-7-7b327ceb6879>, line 2)

In [ ]:
#하이퍼 파라미터 dictionary
params={
    
}